In [1]:
import egglib
import glob 
import pandas as pd
import numpy as np
import os
import seaborn as sns

Goal: calculate nucleotide diversity per site for NLR clades using the egglib python package 

In [2]:
#gather alignments in fasta format 
directory='/global/scratch/users/chandlersutherland/e14/popgen/popgenome_test'
files = glob.glob(os.path.join(directory, "*.fas"))
#import nlr annotations generated by nlr_annotation.ipynb 
annotations=pd.read_csv('/global/scratch/users/chandlersutherland/e14/popgen/nlr_aa_annotation.csv')[['clade', 'start', 'end', 'domain']]

In [48]:
#write a function that takes in protein alignment coordinates and clade and outputs the desired stats 
def sub_stat(row):
    #load alignment, and subset by domain
    fasta='/global/scratch/users/chandlersutherland/e14/popgen/popgenome_test/'+row[0]+'.pal2nal.fas'
    clade=row[0]
    domain=row[3]
    aln=egglib.io.from_fasta(fasta, alphabet=egglib.alphabets.DNA)
    codon=egglib.tools.to_codons(aln)
    subset=codon.extract(int(row[1]), int(row[2]))
    
    #compute across subset stats
    dna=egglib.tools.to_bases(subset)
    ls=egglib.stats.ComputeStats()
    ls.configure(multi_hits=True, multi=False)
    ls.add_stats('Pi', 'lseff', 'D')
    stats=ls.process_align(dna)
    
    lseff=stats['lseff']
    D=stats['D']
    Pi=stats['Pi']
    dna_aln=dna.ls
    
    #compute per synonymous and nonsynonomous site stats 
    cs_subset = egglib.stats.CodingDiversity(subset)

    #use the process_sites function to calculate Pi over just the synonymous and non-synonymous sites 
    statsS=ls.process_sites(cs_subset.sites_S)
    statsNS=ls.process_sites(cs_subset.sites_NS) 

    #normalize by the number of synonymous and nonsynonymous sites
    #if there are no polymorphisms at each site, return NaN
    
    if cs_subset.num_pol_S == 0:
        PiS=np.nan
    else:
        PiS=statsS['Pi']/cs_subset.num_sites_S
    
    if cs_subset.num_pol_NS == 0:
        PiNS=np.nan
    else:
        PiNS=statsNS['Pi']/cs_subset.num_sites_NS
    
    return({'clade':clade, 'domain':domain, 'Pi_raw':Pi, 'lseff':lseff, 'dna_aln':dna_aln, 'PiS':PiS, 'PiN':PiNS, 'D':D})

In [49]:
#check one row 
annotations[annotations['domain']=='cc']

,clade,start,end,domain
699,Int8443_258_453_R_8,28.0,62.0,cc
700,Int8443_258_360_R_43,30.0,64.0,cc
701,Int14642_297_369_R_46,13.0,61.0,cc
702,Int8443_258_351_L_88_120_R_50,29.0,56.0,cc
703,Int7765_208_251_L_43,24.0,159.0,cc
704,Int8443_258_352_R_4,16.0,50.0,cc
705,Int7973_410_746_R_58,36.0,56.0,cc
706,Int14328_59,462.0,529.0,cc
707,Int8443_258_447_R_7,29.0,63.0,cc
708,Int12974_336_392_R_32,1130.0,1157.0,cc


In [6]:
stats=pd.DataFrame()
#calculate per domain stats 
for i in range(0, len(annotations)):
    row=annotations.iloc[i,:]
    result=sub_stat(row)
    result['start']=row[1]
    result['stop']=row[2]
    print(result)
    stats=stats.append(result, ignore_index=True)

stats

{'clade': 'Int11629_24_35_L_12', 'domain': 'nbarc', 'Pi_raw': 5.515151515151515, 'lseff': 708, 'dna_aln': 786, 'PiS': 0.0021551724137934304, 'PiN': 0.00936473165388787, 'D': 0.4737401478402454, 'start': 612.0, 'stop': 874.0}
{'clade': 'Int8443_258_453_R_8', 'domain': 'nbarc', 'Pi_raw': 1.4285714285714286, 'lseff': 840, 'dna_aln': 840, 'PiS': 0.0027431706480744713, 'PiN': 0.0014117838703692233, 'D': -1.1753192292075076, 'start': 157.0, 'stop': 437.0}
{'clade': 'Int9156_236_297_R_14', 'domain': 'nbarc', 'Pi_raw': 0.0, 'lseff': 753, 'dna_aln': 753, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 201.0, 'stop': 452.0}
{'clade': 'Int9687_297_427_R_29', 'domain': 'nbarc', 'Pi_raw': 9.147783251231527, 'lseff': 696, 'dna_aln': 741, 'PiS': 0.01715561224490036, 'PiN': 0.009925489835341318, 'D': 1.0235829317780445, 'start': 201.0, 'stop': 448.0}
{'clade': 'Int12133_61', 'domain': 'nbarc', 'Pi_raw': 19.571584699453528, 'lseff': 732, 'dna_aln': 774, 'PiS': 0.03028631076648175, 'PiN': 0.0163867084231498

{'clade': 'Int12974_336_392_R_32', 'domain': 'nbarc', 'Pi_raw': 3.5725806451612905, 'lseff': 609, 'dna_aln': 609, 'PiS': 0.01666441998815024, 'PiN': 0.0028748367931194316, 'D': -0.13573026580116535, 'start': 192.0, 'stop': 395.0}
{'clade': 'Int9687_297_319_L_40', 'domain': 'nbarc', 'Pi_raw': 2.573076923076922, 'lseff': 741, 'dna_aln': 762, 'PiS': 0.0041572579452344, 'PiN': 0.003123372663248733, 'D': -1.7601474363552247, 'start': 224.0, 'stop': 478.0}
{'clade': 'Int13684_55_74_R_32', 'domain': 'nbarc', 'Pi_raw': 5.59274193548387, 'lseff': 753, 'dna_aln': 810, 'PiS': 0.019673286663998875, 'PiN': 0.004118484946217467, 'D': 0.43425120664917033, 'start': 215.0, 'stop': 485.0}
{'clade': 'Int14642_297_350_R_17', 'domain': 'nbarc', 'Pi_raw': 2.014705882352941, 'lseff': 777, 'dna_aln': 852, 'PiS': 0.00634088603314261, 'PiN': 0.0015119001170502675, 'D': -0.09319433425466839, 'start': 169.0, 'stop': 453.0}
{'clade': 'Int14174_152_193_R_60', 'domain': 'nbarc', 'Pi_raw': 1.6627118644067787, 'lseff'

{'clade': 'Int15268_115_157_L_57', 'domain': 'nbarc', 'Pi_raw': 10.447994987468672, 'lseff': 834, 'dna_aln': 855, 'PiS': 0.028248710432237884, 'PiN': 0.005329968334338446, 'D': 1.0044719504954764, 'start': 189.0, 'stop': 474.0}
{'clade': 'Int14939_40_64_L_31', 'domain': 'nbarc', 'Pi_raw': 57.638709677419364, 'lseff': 744, 'dna_aln': 891, 'PiS': 0.07319570039243406, 'PiN': 0.030222805332332577, 'D': 0.815478976460326, 'start': 175.0, 'stop': 472.0}
{'clade': 'Int12715_16', 'domain': 'nbarc', 'Pi_raw': 14.750000000000007, 'lseff': 726, 'dna_aln': 780, 'PiS': 0.03177745084927544, 'PiN': 0.011733532375442722, 'D': 0.8141767445873778, 'start': 211.0, 'stop': 471.0}
{'clade': 'Int14387_212_286_R_29', 'domain': 'nbarc', 'Pi_raw': 7.822660098522162, 'lseff': 699, 'dna_aln': 864, 'PiS': 0.006751304229227028, 'PiN': 0.006982715024831466, 'D': -1.8101521610945634, 'start': 168.0, 'stop': 456.0}
{'clade': 'Int7764_78_107_R_21', 'domain': 'nbarc', 'Pi_raw': 4.8095238095238075, 'lseff': 834, 'dna_al

{'clade': 'Int10172_376_624_R_44', 'domain': 'nbarc', 'Pi_raw': 0.8403805496828749, 'lseff': 777, 'dna_aln': 789, 'PiS': 0.0015354766895707577, 'PiN': 0.0008094502294461772, 'D': -2.1040564632702123, 'start': 717.0, 'stop': 980.0}
{'clade': 'Int14979_63_105_L_46', 'domain': 'nbarc', 'Pi_raw': 3.3719806763285054, 'lseff': 666, 'dna_aln': 852, 'PiS': 0.005076931046988156, 'PiN': 0.002814183102541697, 'D': -2.0109985280675327, 'start': 168.0, 'stop': 452.0}
{'clade': 'Int12974_336_507_R_56', 'domain': 'nbarc', 'Pi_raw': 5.2227272727272736, 'lseff': 711, 'dna_aln': 711, 'PiS': 0.008475223333589316, 'PiN': 0.005283895039548131, 'D': -0.0011761569678897493, 'start': 174.0, 'stop': 411.0}
{'clade': 'Int7973_410_712_R_35', 'domain': 'nbarc', 'Pi_raw': 1.4806722689075633, 'lseff': 489, 'dna_aln': 843, 'PiS': 0.0010685375067459647, 'PiN': 0.0004785961180536897, 'D': -2.573317125558081, 'start': 175.0, 'stop': 456.0}
{'clade': 'Int8760_26_29_L_4', 'domain': 'nbarc', 'Pi_raw': 0.5, 'lseff': 825, '

{'clade': 'Int11247_267_468_R_60', 'domain': 'nbarc', 'Pi_raw': 12.868361581920896, 'lseff': 765, 'dna_aln': 768, 'PiS': 0.040763745593562314, 'PiN': 0.005430444687128767, 'D': -1.2304766161256921, 'start': 206.0, 'stop': 462.0}
{'clade': 'Int8908_92_169_R_11', 'domain': 'nbarc', 'Pi_raw': 0.7272727272727273, 'lseff': 828, 'dna_aln': 828, 'PiS': 0.002008536279186819, 'PiN': 0.0005620740532564946, 'D': -1.7116624245540921, 'start': 150.0, 'stop': 426.0}
{'clade': 'Int13909_42', 'domain': 'nbarc', 'Pi_raw': 11.851335656213681, 'lseff': 756, 'dna_aln': 759, 'PiS': 0.02316624693828672, 'PiN': 0.004448257895268712, 'D': -2.1654542477976535, 'start': 189.0, 'stop': 442.0}
{'clade': 'Int12730_93_110_L_63', 'domain': 'nbarc', 'Pi_raw': 5.143881208397337, 'lseff': 723, 'dna_aln': 816, 'PiS': 0.017410852007248588, 'PiN': 0.003933778616503786, 'D': 0.6530180041495548, 'start': 261.0, 'stop': 533.0}
{'clade': 'Int9687_297_408_R_19', 'domain': 'nbarc', 'Pi_raw': 2.0350877192982457, 'lseff': 744, 'd

...

{'clade': 'Int12133_61', 'domain': 'tir', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 576, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 55.0, 'stop': 247.0}
{'clade': 'Int9687_297_557_R_19', 'domain': 'tir', 'Pi_raw': 1.9649122807017558, 'lseff': 444, 'dna_aln': 615, 'PiS': 0.006628613515007372, 'PiN': 0.003247741804526414, 'D': -2.2580696235566626, 'start': 15.0, 'stop': 220.0}
{'clade': 'Int11247_267_326_R_108_147_R_52_64_R_9', 'domain': 'tir', 'Pi_raw': 12.83333333333333, 'lseff': 501, 'dna_aln': 516, 'PiS': 0.023635031146944846, 'PiN': 0.008789353461117814, 'D': 0.6284346776658895, 'start': 69.0, 'stop': 241.0}
{'clade': 'Int10637_304_324_L_31', 'domain': 'tir', 'Pi_raw': 1.9118279569892465, 'lseff': 174, 'dna_aln': 504, 'PiS': 0.00945989133908756, 'PiN': 0.0010810810810810293, 'D': -2.3454790313754255, 'start': 12.0, 'stop': 180.0}
{'clade': 'Int9983_190_234_L_53', 'domain': 'tir', 'Pi_raw': 1.6444121915820027, 'lseff': 33, 'dna_aln': 552, 'PiS': nan, 'PiN': 0.025444241100418336, 'D': 0

{'clade': 'Int14006_29_44_R_10', 'domain': 'tir', 'Pi_raw': 0.9555555555555555, 'lseff': 537, 'dna_aln': 537, 'PiS': 0.003462703794546866, 'PiN': 0.001318096141932525, 'D': -0.35590560554997647, 'start': 12.0, 'stop': 191.0}
{'clade': 'Int9687_297_380_R_42', 'domain': 'tir', 'Pi_raw': 0.6341463414634145, 'lseff': 81, 'dna_aln': 573, 'PiS': 0.030871490245010066, 'PiN': 0.0014639636205039835, 'D': -0.19476279246775058, 'start': 13.0, 'stop': 204.0}
{'clade': 'Int10547_34_50_R_17', 'domain': 'tir', 'Pi_raw': 0.7205882352941176, 'lseff': 393, 'dna_aln': 486, 'PiS': 0.007208157524614484, 'PiN': 0.00038030043734548456, 'D': -0.5323441879594113, 'start': 32.0, 'stop': 194.0}
{'clade': 'Int11170_39_55_L_37', 'domain': 'tir', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 513, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 20.0, 'stop': 191.0}
{'clade': 'Int10580_55', 'domain': 'tir', 'Pi_raw': 2.0942760942760943, 'lseff': 78, 'dna_aln': 705, 'PiS': nan, 'PiN': 0.032985207545423154, 'D': 0.530145380293353

{'clade': 'Int9687_297_382_R_27', 'domain': 'tir', 'Pi_raw': 3.2079772079772093, 'lseff': 123, 'dna_aln': 534, 'PiS': 0.017621906448410624, 'PiN': 0.008545582857509152, 'D': -2.1658680074569325, 'start': 13.0, 'stop': 191.0}
{'clade': 'Int12730_93_111_R_29', 'domain': 'tir', 'Pi_raw': 3.221674876847291, 'lseff': 477, 'dna_aln': 522, 'PiS': 0.005100017000057511, 'PiN': 0.006224605630579805, 'D': -0.8810239619416128, 'start': 20.0, 'stop': 194.0}
{'clade': 'Int9983_190_284_R_29', 'domain': 'tir', 'Pi_raw': 0.7635467980295564, 'lseff': 72, 'dna_aln': 645, 'PiS': 0.0289710289710343, 'PiN': 0.005855506759665611, 'D': -1.1016758738117158, 'start': 13.0, 'stop': 228.0}
{'clade': 'Int11629_24_36_R_11', 'domain': 'tir', 'Pi_raw': 2.727272727272727, 'lseff': 21, 'dna_aln': 531, 'PiS': 0.27272727272754543, 'PiN': nan, 'D': -2.072516832581659, 'start': 60.0, 'stop': 237.0}
{'clade': 'Int11093_76_90_L_19', 'domain': 'tir', 'Pi_raw': 0.6315789473684215, 'lseff': 477, 'dna_aln': 579, 'PiS': 0.0063252

{'clade': 'Int8443_258_351_L_88_120_R_50', 'domain': 'cc', 'Pi_raw': 2.278367346938776, 'lseff': 75, 'dna_aln': 81, 'PiS': 0.003232758620690521, 'PiN': 0.011853441705333784, 'D': -1.362670842655233, 'start': 29.0, 'stop': 56.0}
{'clade': 'Int7765_208_251_L_43', 'domain': 'cc', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 405, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 24.0, 'stop': 159.0}
{'clade': 'Int8443_258_352_R_4', 'domain': 'cc', 'Pi_raw': 0.0, 'lseff': 102, 'dna_aln': 102, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 16.0, 'stop': 50.0}
{'clade': 'Int7973_410_746_R_58', 'domain': 'cc', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 60, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 36.0, 'stop': 56.0}
{'clade': 'Int14328_59', 'domain': 'cc', 'Pi_raw': 0.800116890707189, 'lseff': 108, 'dna_aln': 201, 'PiS': 0.0016400164001642936, 'PiN': 0.0009610156417940142, 'D': -2.377167931940818, 'start': 462.0, 'stop': 529.0}
{'clade': 'Int8443_258_447_R_7', 'domain': 'cc', 'Pi_raw': 0.0, 'lseff': 102, 'dna

,D,PiN,PiS,Pi_raw,clade,dna_aln,domain,lseff,start,stop
0,0.47374,0.009365,0.002155,5.51515,Int11629_24_35_L_12,786.0,nbarc,708.0,612.0,874.0
1,-1.17532,0.001412,0.002743,1.42857,Int8443_258_453_R_8,840.0,nbarc,840.0,157.0,437.0
2,None,NaN,NaN,0,Int9156_236_297_R_14,753.0,nbarc,753.0,201.0,452.0
3,1.02358,0.009925,0.017156,9.14778,Int9687_297_427_R_29,741.0,nbarc,696.0,201.0,448.0
4,-0.0523929,0.016387,0.030286,19.5716,Int12133_61,774.0,nbarc,732.0,250.0,508.0
...,...,...,...,...,...,...,...,...,...,...
742,0.093291,0.010682,0.014905,1.95257,Int15041_179_288_R_23,120.0,cc,120.0,17.0,57.0
743,None,NaN,NaN,None,Int14642_297_414_R_121,150.0,cc,0.0,133.0,183.0
744,None,NaN,NaN,None,Int7973_410_504_R_207,102.0,cc,0.0,113.0,147.0
745,None,NaN,NaN,None,Int14642_297_534_R_55,156.0,cc,0.0,13.0,65.0


In [67]:
#add Col-0 gene names and HV label
gene_table=pd.read_table('/global/scratch/users/chandlersutherland/e14/popgen/Atha_NLRome_GeneTable.txt')[['Gene', 'Clade', 'Ecotype', 'HV']]
gene_table=gene_table[gene_table['Ecotype']=='ATHALIANA']
gene_table['Gene']=gene_table['Gene'].str.split('_', expand=True)[1]
gene_table=gene_table.rename(columns={'Clade':'clade'})[['Gene', 'clade', 'HV']]
gene_table=gene_table.assign(HV=gene_table.HV.map({0:'non-hv', 1:'hv'}))

#egglib_results=pd.merge(stats, gene_table)
#egglib_results

In [36]:
egglib_results[egglib_results['domain']=='cc']

,D,PiN,PiS,Pi_raw,clade,dna_aln,domain,lseff,start,stop,Gene,HV
15,None,NaN,NaN,None,Int14642_297_369_R_46,144.0,cc,0.0,13.0,61.0,AT5G35450,non-hv
19,-1.36267,0.011853,0.003233,2.27837,Int8443_258_351_L_88_120_R_50,81.0,cc,75.0,29.0,56.0,AT5G43740,hv
23,None,NaN,NaN,None,Int7765_208_251_L_43,405.0,cc,0.0,24.0,159.0,AT1G63350,non-hv
57,-2.37717,0.000961,0.001640,0.800117,Int14328_59,201.0,cc,108.0,462.0,529.0,AT3G50950,non-hv
80,None,NaN,NaN,0,Int8443_258_447_R_7,102.0,cc,102.0,29.0,63.0,AT1G15890,non-hv
95,-2.42794,NaN,NaN,4.52016,Int12974_336_392_R_32,81.0,cc,75.0,1130.0,1157.0,AT5G17890,non-hv
143,None,NaN,NaN,None,Int15041_179_310_R_37,87.0,cc,0.0,22.0,51.0,AT1G58390,non-hv
150,None,NaN,NaN,None,Int7765_208_274_R_142,153.0,cc,0.0,38.0,89.0,AT1G63360,hv
151,None,NaN,NaN,None,Int7765_208_274_R_142,153.0,cc,0.0,38.0,89.0,AT1G62630,hv
156,-1.19598,0.005935,NaN,0.372414,Int13629_56_82_R_30,81.0,cc,81.0,247.0,274.0,AT4G19530,non-hv


In [33]:
egglib_results.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/egglib_results.tsv', sep='\t')

Complement the per domain analysis with sliding window

In [5]:
cds_only=annotations[annotations['domain']=='cds']

In [50]:
#function that takes in a clade name, and evaluates Pi, PiS, PiNS, and D across sliding windows of window size 33 codons and step size 10 codons 
#outputs a dataframe with the results across the codon windows 
def my_slider(row, window, step):
    #get codon length 
    codon_length=row[2]
    clade=row[0]
    
    #generate a new annotations dataframe with sliding windows to this length 
    fake_annotations=pd.DataFrame()
    for i in range(0, int(codon_length/step)):
        fake_annotations=fake_annotations.append({'start':0+i*step, 'end':window+i*step, 'clade':clade, 'domain':'window'}, ignore_index=True)
    fake_annotations=fake_annotations[['clade', 'start', 'end', 'domain']]
    return fake_annotations

In [51]:
#create new annotations df with sliding windows
slide=[]
for i in range(0, len(cds_only)):
    row=cds_only.iloc[i,:]
    windows=my_slider(row, 100, 25)
    slide.append(windows)

slide_df=pd.concat(slide)
slide_df

,clade,start,end,domain
0,Int11629_24_35_L_12,0.0,100.0,window
1,Int11629_24_35_L_12,25.0,125.0,window
2,Int11629_24_35_L_12,50.0,150.0,window
3,Int11629_24_35_L_12,75.0,175.0,window
4,Int11629_24_35_L_12,100.0,200.0,window
...,...,...,...,...
100,Int12432_284,2500.0,2600.0,window
101,Int12432_284,2525.0,2625.0,window
102,Int12432_284,2550.0,2650.0,window
103,Int12432_284,2575.0,2675.0,window


In [ ]:
window_stats=pd.DataFrame()
for i in range(0, len(slide_df)):
    row=slide_df.iloc[i,:]
    result=sub_stat(row)
    result['start']=row[1]
    result['stop']=row[2]
    print(result)
    window_stats=window_stats.append(result, ignore_index=True)

window_stats

{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 5.5606060606060606, 'lseff': 78, 'dna_aln': 300, 'PiS': 0.14787485242033102, 'PiN': 0.041877508288254844, 'D': -0.2924365218952695, 'start': 0.0, 'stop': 100.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 5.5606060606060606, 'lseff': 78, 'dna_aln': 300, 'PiS': 0.14787485242033102, 'PiN': 0.041877508288254844, 'D': -0.2924365218952695, 'start': 25.0, 'stop': 125.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 1.3181818181818183, 'lseff': 27, 'dna_aln': 300, 'PiS': 0.08269794721407252, 'PiN': 0.03295797857731462, 'D': -0.016928012585064142, 'start': 50.0, 'stop': 150.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 75.0, 'stop': 175.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 100.0, 'stop': 200.0}


{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 0.16666666666666669, 'lseff': 189, 'dna_aln': 300, 'PiS': nan, 'PiN': 0.0011463507833396416, 'D': -1.140533404699156, 'start': 1075.0, 'stop': 1175.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 0.16666666666666669, 'lseff': 264, 'dna_aln': 300, 'PiS': nan, 'PiN': 0.0008214676889375334, 'D': -1.140533404699156, 'start': 1100.0, 'stop': 1200.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 0.16666666666666669, 'lseff': 300, 'dna_aln': 300, 'PiS': nan, 'PiN': 0.0007345739471106478, 'D': -1.140533404699156, 'start': 1125.0, 'stop': 1225.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 0.0, 'lseff': 300, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1150.0, 'stop': 1250.0}
{'clade': 'Int11629_24_35_L_12', 'domain': 'window', 'Pi_raw': 0.4848484848484848, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.006479044340959996, 'PiN': nan, 'D': 1.0658947179863445, 'start': 1175

{'clade': 'Int8443_258_453_R_8', 'domain': 'window', 'Pi_raw': 0.0, 'lseff': 300, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 625.0, 'stop': 725.0}
{'clade': 'Int8443_258_453_R_8', 'domain': 'window', 'Pi_raw': 0.0, 'lseff': 300, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 650.0, 'stop': 750.0}
{'clade': 'Int8443_258_453_R_8', 'domain': 'window', 'Pi_raw': 0.0, 'lseff': 300, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 675.0, 'stop': 775.0}
{'clade': 'Int8443_258_453_R_8', 'domain': 'window', 'Pi_raw': 0.25, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.003685503685504136, 'PiN': nan, 'D': -1.0548191070623705, 'start': 700.0, 'stop': 800.0}
{'clade': 'Int8443_258_453_R_8', 'domain': 'window', 'Pi_raw': 0.5, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.0036057692307697145, 'PiN': 0.0010838150289016904, 'D': -1.3100922360361853, 'start': 725.0, 'stop': 825.0}
{'clade': 'Int8443_258_453_R_8', 'domain': 'window', 'Pi_raw': 0.5, 'lseff': 300, 'dna_aln': 300

{'clade': 'Int9156_236_297_R_14', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 975.0, 'stop': 1075.0}
{'clade': 'Int9156_236_297_R_14', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1000.0, 'stop': 1100.0}
{'clade': 'Int9156_236_297_R_14', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1025.0, 'stop': 1125.0}
{'clade': 'Int9156_236_297_R_14', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1050.0, 'stop': 1150.0}
{'clade': 'Int9156_236_297_R_14', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1075.0, 'stop': 1175.0}
{'clade': 'Int9156_236_297_R_14', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1100.0, 'stop': 1200.0}
{'cla

{'clade': 'Int9687_297_427_R_29', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 875.0, 'stop': 975.0}
{'clade': 'Int9687_297_427_R_29', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 900.0, 'stop': 1000.0}
{'clade': 'Int9687_297_427_R_29', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 925.0, 'stop': 1025.0}
{'clade': 'Int9687_297_427_R_29', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 950.0, 'stop': 1050.0}
{'clade': 'Int9687_297_427_R_29', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 975.0, 'stop': 1075.0}
{'clade': 'Int9687_297_427_R_29', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1000.0, 'stop': 1100.0}
{'clade': 

...

{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': 7.990960451977401, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.05904350585797557, 'PiN': 0.010617209616312122, 'D': 2.177684417856628, 'start': 100.0, 'stop': 200.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': 5.818079096045198, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.03852066594142129, 'PiN': 0.006905195179912283, 'D': 2.7626687340154095, 'start': 125.0, 'stop': 225.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': 4.048587570621469, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.023545372392333123, 'PiN': 0.003394837282884453, 'D': 2.458992245090626, 'start': 150.0, 'stop': 250.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': 1.694915254237288, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.02016187416455933, 'PiN': 0.0014519347090697265, 'D': 0.329963941488386, 'start': 175.0, 'stop': 275.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': 2.3790960451977394, 'lseff':

{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1225.0, 'stop': 1325.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1250.0, 'stop': 1350.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1275.0, 'stop': 1375.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1300.0, 'stop': 1400.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1325.0, 'stop': 1425.0}
{'clade': 'Int12974_336_448_R_60', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1350.0, 'stop': 1450.0

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': 24.53226357118471, 'lseff': 300, 'dna_aln': 300, 'PiS': 0.044131374710934194, 'PiN': 0.03584772223852991, 'D': 1.4643152552283412, 'start': 875.0, 'stop': 975.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': 26.74642735209483, 'lseff': 252, 'dna_aln': 300, 'PiS': 0.06587042358273275, 'PiN': 0.043848385100649194, 'D': 1.5928677456242073, 'start': 900.0, 'stop': 1000.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': 21.47988099302542, 'lseff': 177, 'dna_aln': 300, 'PiS': 0.104885151961373, 'PiN': 0.03331076402479275, 'D': 1.497670203814399, 'start': 925.0, 'stop': 1025.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': 14.699604935863052, 'lseff': 102, 'dna_aln': 300, 'PiS': 0.12177760445629777, 'PiN': 0.044294614827335425, 'D': 1.9787549699801965, 'start': 950.0, 'stop': 1050.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': 4.544066721943131, 'ls

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2050.0, 'stop': 2150.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2075.0, 'stop': 2175.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2100.0, 'stop': 2200.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2125.0, 'stop': 2225.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2150.0, 'stop': 2250.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2175.0, 'stop': 

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3275.0, 'stop': 3375.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3300.0, 'stop': 3400.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3325.0, 'stop': 3425.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3350.0, 'stop': 3450.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3375.0, 'stop': 3475.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3400.0, 'stop': 

In [ ]:
egglib_window_results=pd.merge(window_stats, gene_table)
egglib_window_results

In [46]:
partial=pd.read_table('/global/scratch/users/chandlersutherland/e14/popgen/slider_partial.csv', sep=',')
pick_up=partial.iloc[-1,5]
#slide_df=slide_df.reset_index()
slide_df
#slide_df['clade']==pick_up
slide_df.index[slide_df['clade'] == pick_up].tolist()[0]
#pick_up
#slide_df[slide_df['clade']==pick_up]

8656

In [52]:
window_stats=pd.DataFrame()
for i in range(8656, len(slide_df)):
    row=slide_df.iloc[i,:]
    result=sub_stat(row)
    result['start']=row[1]
    result['stop']=row[2]
    print(result)
    window_stats=window_stats.append(result, ignore_index=True)

window_stats

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 0.0, 'stop': 100.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 25.0, 'stop': 125.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 50.0, 'stop': 150.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 75.0, 'stop': 175.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 100.0, 'stop': 200.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 125.0, 'stop': 225.0}
{'clade':

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1125.0, 'stop': 1225.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1150.0, 'stop': 1250.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1175.0, 'stop': 1275.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1200.0, 'stop': 1300.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1225.0, 'stop': 1325.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1250.0, 'stop': 

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2350.0, 'stop': 2450.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2375.0, 'stop': 2475.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2400.0, 'stop': 2500.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2425.0, 'stop': 2525.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2450.0, 'stop': 2550.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2475.0, 'stop': 

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3575.0, 'stop': 3675.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3600.0, 'stop': 3700.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3625.0, 'stop': 3725.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3650.0, 'stop': 3750.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3675.0, 'stop': 3775.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 3700.0, 'stop': 

{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 4800.0, 'stop': 4900.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 4825.0, 'stop': 4925.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 4850.0, 'stop': 4950.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 4875.0, 'stop': 4975.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 4900.0, 'stop': 5000.0}
{'clade': 'Int10637_304_324_R_203', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 4925.0, 'stop': 

...

{'clade': 'Int11247_267_434_R_35', 'domain': 'window', 'Pi_raw': 11.79663865546219, 'lseff': 213, 'dna_aln': 300, 'PiS': 0.00888109204539397, 'PiN': 0.03309657616512527, 'D': 1.6730395423692652, 'start': 650.0, 'stop': 750.0}
{'clade': 'Int11247_267_434_R_35', 'domain': 'window', 'Pi_raw': 7.6689075630252095, 'lseff': 228, 'dna_aln': 300, 'PiS': 0.012243648607287019, 'PiN': 0.026635544710023957, 'D': 0.5917878922143427, 'start': 675.0, 'stop': 775.0}
{'clade': 'Int11247_267_434_R_35', 'domain': 'window', 'Pi_raw': 7.423529411764705, 'lseff': 219, 'dna_aln': 300, 'PiS': 0.01616115196078721, 'PiN': 0.020410128044670205, 'D': 0.46127930433542885, 'start': 700.0, 'stop': 800.0}
{'clade': 'Int11247_267_434_R_35', 'domain': 'window', 'Pi_raw': 4.815126050420168, 'lseff': 144, 'dna_aln': 300, 'PiS': 0.02478222570226443, 'PiN': 0.013159950887375297, 'D': 0.14699587570026254, 'start': 725.0, 'stop': 825.0}
{'clade': 'Int11247_267_434_R_35', 'domain': 'window', 'Pi_raw': 8.524369747899161, 'lsef

{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 50.0, 'stop': 150.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 75.0, 'stop': 175.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 100.0, 'stop': 200.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 125.0, 'stop': 225.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 150.0, 'stop': 250.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 175.0, 'stop': 275.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lse

{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1375.0, 'stop': 1475.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 1400.0, 'stop': 1500.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': 5.336559996018514, 'lseff': 51, 'dna_aln': 300, 'PiS': 0.15894891745900006, 'PiN': 0.04620646543013807, 'D': 1.021455398900364, 'start': 1425.0, 'stop': 1525.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': 15.923505698501966, 'lseff': 111, 'dna_aln': 300, 'PiS': 0.19161857079324351, 'PiN': 0.02665218871934064, 'D': 1.6137678063402692, 'start': 1450.0, 'stop': 1550.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': 25.85171452744737, 'lseff': 177, 'dna_aln': 300, 'PiS': 0.21996620780351706, 'PiN': 0.02345741332784838, 'D': 1.441141151423838, 'start': 1475.0, 'stop': 1575.0}
{'clade': 'Int12432_284', 'do

{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 300, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2525.0, 'stop': 2625.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 264, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2550.0, 'stop': 2650.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 189, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2575.0, 'stop': 2675.0}
{'clade': 'Int12432_284', 'domain': 'window', 'Pi_raw': None, 'lseff': 0, 'dna_aln': 114, 'PiS': nan, 'PiN': nan, 'D': None, 'start': 2600.0, 'stop': 2700.0}


,D,PiN,PiS,Pi_raw,clade,dna_aln,domain,lseff,start,stop
0,None,NaN,NaN,None,Int10637_304_324_R_203,300.0,window,0.0,0.0,100.0
1,None,NaN,NaN,None,Int10637_304_324_R_203,300.0,window,0.0,25.0,125.0
2,None,NaN,NaN,None,Int10637_304_324_R_203,300.0,window,0.0,50.0,150.0
3,None,NaN,NaN,None,Int10637_304_324_R_203,300.0,window,0.0,75.0,175.0
4,None,NaN,NaN,None,Int10637_304_324_R_203,300.0,window,0.0,100.0,200.0
...,...,...,...,...,...,...,...,...,...,...
686,None,NaN,NaN,None,Int12432_284,300.0,window,0.0,2500.0,2600.0
687,None,NaN,NaN,None,Int12432_284,300.0,window,0.0,2525.0,2625.0
688,None,NaN,NaN,None,Int12432_284,264.0,window,0.0,2550.0,2650.0
689,None,NaN,NaN,None,Int12432_284,189.0,window,0.0,2575.0,2675.0


In [64]:
window_stats['D'].mean()
partial=partial[partial['clade']!=pick_up][['D', 'PiN', 'PiS', 'Pi_raw', 'clade', 'dna_aln', 'domain', 'lseff', 'start', 'stop']]
partial
egglib_window_results=pd.concat([partial, window_stats]).reset_index(drop=True)
egglib_window_results

,D,PiN,PiS,Pi_raw,clade,dna_aln,domain,lseff,start,stop
0,-0.292437,0.041878,0.147875,5.56061,Int11629_24_35_L_12,300.0,window,78.0,0.0,100.0
1,-0.292437,0.041878,0.147875,5.56061,Int11629_24_35_L_12,300.0,window,78.0,25.0,125.0
2,-0.016928,0.032958,0.082698,1.31818,Int11629_24_35_L_12,300.0,window,27.0,50.0,150.0
3,NaN,NaN,NaN,NaN,Int11629_24_35_L_12,300.0,window,0.0,75.0,175.0
4,NaN,NaN,NaN,NaN,Int11629_24_35_L_12,300.0,window,0.0,100.0,200.0
...,...,...,...,...,...,...,...,...,...,...
9342,None,NaN,NaN,None,Int12432_284,300.0,window,0.0,2500.0,2600.0
9343,None,NaN,NaN,None,Int12432_284,300.0,window,0.0,2525.0,2625.0
9344,None,NaN,NaN,None,Int12432_284,264.0,window,0.0,2550.0,2650.0
9345,None,NaN,NaN,None,Int12432_284,189.0,window,0.0,2575.0,2675.0


In [69]:
egglib_window_final=pd.merge(egglib_window_results, gene_table)
egglib_window_final.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/egglib_window_stats.csv')